# 3.1 Parabolic model problem
In this unit we want to turn to instationary problems. And we will start with a very basic setup: implicit Euler time stepping for the convection diffusion equation. After the main part of this tutorial we have supplementary material to extend the case in several regards.


The Strong form:

$$\partial_tu - \Delta u= f \mbox{ in } \Omega \times [0,T]$$
$$u(\cdot, t=0) = u_0 \mbox{ in } \Omega $$
$$u=0 \mbox{ on } \partial \Omega \times [0,T]$$

We are solving the unsteady heat equation 

$$\text{find } u:[0,T] \to H_{0,D}^1 \quad \int_{\Omega} \partial_t u v + \int_{\Omega} \nabla u \nabla v = \int f v  \quad \forall v \in H_{0,D}^1, \quad u(t=0) = u_0$$

with a suitable advective field $b$ (the wind).


## Variational formulation in space

Similar as for elliptic equations we pass over to the weak formulations. We do this only for the spatial variables $x$, for all $t \in (0,T)$. The test functions $v$ depend only on $x$. We multiply by $v$, integration over $\Omega$, and perform integration by parts to obtain:

$$
\int_\Omega \frac{\partial u}{\partial t}(x,t) v(x) \, dx + \int \nabla u(x,t) \nabla v(x) \, dx = \int_\Omega f(x,t) v(x) \, dx \qquad \forall \, v \in H_0^1(\Omega) \; \forall t \, \in (0,T)
$$

The initial condition in $L_2$ is set for $t = 0$:
$$
u(\cdot, 0) = u_0 \qquad \text{in} \; L_2(\Omega)
$$

We consider the unknown function $u$ as a function with values in the Hilbert space $H^1$, e.g. $u : [0,T] \rightarrow H^1(\Omega)$. This allows to express the equation as Hilbert-space valued ordinary differential equation:
$$
\frac{d u }{dt }(t) + A u(t) = f(t)
$$
... some more definitions to come ...

## Galerkin method in space

As a next step we apply the finite element method in space. Let $V_h$ be a finite element sub-space of $H^1(\Omega)$, with the basis functions $\{ p_1(x), \ldots p_N(x) \}$. We approximate the time-dependent function by

$$
u_h : [0,T] \rightarrow V_h,
$$

which can we expressed by means of the basis as

$$
u_h(t) = \sum_{i=1}^N u_i(t) p_i(x)
$$

Inserting this expansion into the variational formulation, and using spatial test-functions $v = p_j$ we obtain the semi-discrete equation:


$$
\int_\Omega \frac{\partial}{\partial t} \Big\{ \sum_{i=1}^N u_i(t) p_i(x) \Big\} p_j(x) dx + 
\int_\Omega \nabla \Big\{ \sum_{i=1}^N u_i(t) p_i(x) \Big\} \; \nabla p_j(x) = \int_\Omega f(x,t) p_j(x) \, dx
\qquad \forall \, j = 1 \ldots N, \forall \, t \in (0,T)
$$


The initial condition for $u_h$ is obtained by some kind of projection, typically we want the $L_2$ projection

$$
\int u_h(x,0) p_j(x) \, dx = \int u_0(x) p_j(x) \, dx \qquad \forall \, j \in \{ 1, \ldots N \}
$$





By means of the **mass matrix** $M \in R^{N \times N}$ 
$$
M = \left\{ \int_\Omega p_i p_j \, dx \right\}_{i,j = 1, \ldots N}
$$
and **stiffness matrix** $A \in R^{N \times N}$ 
$$
A = \left\{ \int_\Omega \nabla p_i \nabla p_j \, dx \right\}_{i,j = 1, \ldots N}
$$
we obtain the ordinary differential equation (ODE)

$$
M \dot u(t) + A u(t) = f(t) \qquad \forall \, t 
$$

including the initial condition 

$$
u(0) = u_0
$$

for the unknown coefficient function $u(\cdot) = (u_1(\cdot), \ldots u_N(\cdot)) \in C^1([0,T], R^N)$.

### Data

$$
u(x,y,t) = e^{-t}xy(x-1)(y-1)\\
u_0=xy(x-1)(y-1)\\
\Delta t=0.001\\
$$
NB: We evaluate the given source f using $f=\partial_t u - \Delta u$.

In [ ]:
#imports
from ngsolve import *
from netgen.geom2d import SplineGeometry
from ngsolve.webgui import Draw

* Geometry: $(-1,1)^2$
* Dirichlet boundaries everywhere
* Mesh

In [ ]:
from netgen.occ import *
from netgen.webgui import Draw as DrawGeo
shape = Rectangle(1,1).Face()
shape.edges.Min(X).name="left"
shape.edges.Max(X).name="right"
shape.edges.Min(Y).name="bottom"
shape.edges.Max(Y).name="top"


In [ ]:
mesh = Mesh(OCCGeometry(shape, dim=2).GenerateMesh(maxh=0.25))
fes = H1(mesh, order=3, dirichlet="bottom|right|left|top")
u,v = fes.TnT()
time = 0.0
dt = 0.001

In [ ]:
u0 = x*y*(x-1)*(y-1)
t = Parameter(0)
exact = exp(-t)*x*y*(x-1)*(y-1)
source = exact.Diff(t) - exact.Diff(x).Diff(x) - exact.Diff(y).Diff(y)

* bilinear forms for 
 * convection-diffusion stiffness and 
 * mass matrix separately.
* non-symmetric memory layout for the mass matrix so that a and m have the same sparsity pattern.

In [ ]:
a = BilinearForm(fes, symmetric=False)
a += grad(u)*grad(v)*dx
a.Assemble()

m = BilinearForm(fes, symmetric=False)
m += u*v*dx
m.Assemble()

## Implicit Euler method
$$
  \underbrace{(M + \Delta t A)}_{M^\ast} u^{n+1} = M u^n + \Delta tf^{n+1}
$$

or in an incremental form:

$$
  M^\ast (u^{n+1} - u^n) = - \Delta t A u^n + \Delta tf^{n+1}.
$$

* Incremental form: $u^{n+1} - u^n$ has homogeneous boundary conditions (unless boundary conditions are time-dependent).
* For the time stepping method: set up linear combinations of matrices.
* (Only) if the sparsity pattern of the matrices agree we can copy the pattern and sum up the entries.

First, we create a matrix of same format as m.mat and compare number of non-zero entries and sparsity pattern:

In [ ]:
mstar = m.mat.CreateMatrix()
print(f"m.mat.nze = {m.mat.nze}, a.mat.nze={a.mat.nze}, mstar.nze={mstar.nze}")

To access the entries we use the vector of nonzero-entries:

In [ ]:
print(f"mstar.nze={mstar.nze}, len(mstar.AsVector())={len(mstar.AsVector())}")

In [ ]:
# print(mstar.AsVector())

Using the vector we can build the linear combination of the a and the m matrix:

In [ ]:
mstar.AsVector().data = m.mat.AsVector() + dt * a.mat.AsVector()
# corresponds to M* = M + dt * A
invmstar = mstar.Inverse(freedofs=fes.FreeDofs())

In [ ]:
gfu = GridFunction(fes)
gfu.Set(u0) # note that boundary conditions remain
scene = Draw(gfu,mesh,"u")

## Supplementary 2: time-dependent r.h.s. data <a class="anchor" id="TDRHS"></a>

Next: time-dependent r.h.s. data $f=f(t)$:

* Use `Parameter` t representing the time. 
* A `Parameter` is a constant `CoefficientFunction` the value of which can be changed with the `Set`-function.

In [ ]:
t = Parameter(0.0)

An example of a time-dependent coefficient that we want to use as r.h.s. in the following is

In [ ]:
gff = GridFunction(L2(mesh,order=gfu.space.globalorder+1))
gfft = GridFunction(gff.space,multidim=0)
time = 0.0
for i in range(7):
    t.Set(3*i/6)
    gff.Set(source)
    gfft.AddMultiDimComponent(gff.vec)
Draw(gfft, mesh, interpolate_multidim=True, animate=True, autoscale=True, min=-1, max=1)

Accordingly, we define a different linear form which then has to be assembled in every time step.

In [ ]:
ft = LinearForm(fes)
ft += source*v*dx

In [ ]:
def TimeStepping_app2(invmstar, initial_cond = None, t0 = 0, tend = 2, 
                      nsamples = 10):
    if initial_cond:
        gfu.Set(initial_cond)
    cnt = 0; time = t0
    sample_int = int(floor(tend / dt / nsamples)+1)
    gfut = GridFunction(gfu.space,multidim=0)
    gfut.AddMultiDimComponent(gfu.vec)
    while time < tend - 0.5 * dt:
        t.Set(time)
        ft.Assemble()
        res = dt * ft.vec - dt * a.mat * gfu.vec
        gfu.vec.data += invmstar * res
        print("\r",time,end="")
        if cnt % sample_int == 0:
            gfut.AddMultiDimComponent(gfu.vec)
        cnt += 1; time = cnt * dt
    return gfut

In [ ]:
%%time
gfut_a2 = TimeStepping_app2(invmstar, initial_cond=u0,tend=2)
Draw(gfut_a2, mesh, interpolate_multidim=True, animate=True, 
     min=-0.75,max=0.75,autoscale=True)

In [ ]:
err = sqrt (Integrate ( (gfut_a2-exact)*(gfut_a2-exact), mesh))
err

In [ ]:
def exact_TimeStepping_app2(invmstar, initial_cond = None, t0 = 0, tend = 2, 
                 nsamples = 10):
    if initial_cond:
        exact_gfu.Set(initial_cond)
    cnt = 0; time = t0
    sample_int = int(floor(tend / dt / nsamples)+1)
    exact_gfut_a2 = GridFunction(exact_gfu.space,multidim=0)
    exact_gfut_a2.AddMultiDimComponent(exact_gfu.vec)
    while time < tend - 0.5 * dt:
        t.Set(time)
        ft.Assemble()
        res = dt * ft.vec - dt * a.mat * exact_gfu.vec
        exact_gfu.vec.data += invmstar * res
        print("\r",time,end="")
        if cnt % sample_int == 0:
            exact_gfut_a2.AddMultiDimComponent(exact_gfu.vec)
        cnt += 1; time = cnt * dt
    return exact_gfut_a2

In [ ]:
mesh_e = Mesh(OCCGeometry(shape, dim=2).GenerateMesh(maxh=0.01))
fes = H1(mesh_e, order=1, dirichlet="bottom|right|left|top")
u,v = fes.TnT()
time = 0.0
dt = 0.001
a = BilinearForm(fes, symmetric=False)
a += grad(u)*grad(v)*dx
a.Assemble()

m = BilinearForm(fes, symmetric=False)
m += u*v*dx
m.Assemble()

mstar = m.mat.CreateMatrix()
mstar.AsVector().data = m.mat.AsVector() + dt * a.mat.AsVector()
# corresponds to M* = M + dt * A
invmstar = mstar.Inverse(freedofs=fes.FreeDofs())
ft = LinearForm(fes)
ft += source*v*dx

exact_gfu = GridFunction(fes)

exact_gfut_a2 = exact_TimeStepping_app2(invmstar, initial_cond=(1-y*y)*x,tend=2)

In [ ]:
Draw(exact_gfut_a2, mesh_e, interpolate_multidim=True, animate=True, 
     min=-0.75,max=0.75,autoscale=False)

In [ ]:
err = sqrt (Integrate ( (gfut_a2-exact_gfut_a2)*(gfut_a2-exact_gfut_a2), mesh))
err

In [ ]:
mesh_size = [0.5, 0.25, 0.125,  0.0625, 0.03125] #, 0.015625]
dofs = []
errors = []

for h in mesh_size:
    mesh = Mesh(OCCGeometry(shape, dim=2).GenerateMesh(maxh=h))
    fes = H1(mesh, order=2, dirichlet="bottom|right|left|top")
    dofs.append(fes.ndof)
    u,v = fes.TnT()
    time = 0.0
    dt = 0.001
    a = BilinearForm(fes, symmetric=False)
    a += grad(u)*grad(v)*dx
    a.Assemble()

    m = BilinearForm(fes, symmetric=False)
    m += u*v*dx
    m.Assemble()

    mstar = m.mat.CreateMatrix()
    mstar.AsVector().data = m.mat.AsVector() + dt * a.mat.AsVector()
    # corresponds to M* = M + dt * A
    invmstar = mstar.Inverse(freedofs=fes.FreeDofs())
    
    ft = LinearForm(fes)
    ft += source*v*dx
    ft.Assemble()

    gfu = GridFunction(fes)

    gfut_a2 = TimeStepping_app2(invmstar, initial_cond=u0,tend=2) 
    Draw(gfut_a2, mesh, interpolate_multidim=True, animate=True, 
     min=-0.75,max=0.75,autoscale=True)
        
#     err = sqrt (Integrate ( (gfut_a2-exact_gfut_a2)*(gfut_a2-exact_gfut_a2), mesh))
    err = sqrt (Integrate ( (gfut_a2-exact)*(gfut_a2-exact), mesh))

    errors.append(err)

In [ ]:
errors

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
for i in range(len(mesh_size)-1):
    print("Convergence rate:", np.log(errors[i+1]/errors[i])/np.log(mesh_size[i+1]/mesh_size[i]))

In [ ]:
n_dofs = [n**(3/2) for n in dofs]
plt.loglog(dofs, np.divide(np.ones(len(dofs)), np.array(n_dofs)), label="Reference Line (1/dofs^(3/2))")
plt.loglog(dofs,errors, "-o", label="L2 Error")
plt.title("Error Estimation Plot")
plt.xlabel("Degrees of Freedom")
plt.ylabel("L2 Error")
plt.legend()
plt.show()

In [ ]:
Draw(gfut, mesh, interpolate_multidim=True, animate=True)